# Input libraries

In [1]:
from IPython.core.interactiveshell import InteractiveShell
import glob
from math import sqrt
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import geopandas as gpd
import geoparquet as gpq
import math
from shapely.geometry import Point

import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import dask_geopandas as dg


from pathlib import Path
import os
import glob

import re
from tqdm.notebook import tqdm

%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

pbar = ProgressBar()
pbar.register()

# Input data

### sa data

In [2]:
%%time
sa = gpd.read_file('data/shapefile/Finn_SA2Summary.shp', crs="EPSG:2193")
sa = sa[['BldgFloo_1', 'PopDensity', 'NumberBldg', 'geometry']] # drop column not used in caluclation of Ignition
sa = gpd.GeoDataFrame(sa)

CPU times: user 183 ms, sys: 24.7 ms, total: 208 ms
Wall time: 386 ms


### pga data

In [3]:
%%time
pga = pd.read_parquet('/Users/alex/Dropbox/Work/Repository/OpenQuake/output/PGAxy.parquet')
pga = gpd.GeoDataFrame(pga, geometry=gpd.points_from_xy(pga.lon, pga.lat, crs="EPSG:2193"))

CPU times: user 5min 24s, sys: 27.3 s, total: 5min 51s
Wall time: 8min 34s


# Merge SA data with PGA events 

In [4]:
%%time
PGA_SA = gpd.sjoin(pga, sa, op="within")

/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2193
Right CRS: PROJCS["NZGD2000_New_Zealand_Transverse_Mercator_2 ...

  """Entry point for launching an IPython kernel.


CPU times: user 29min 45s, sys: 11min 24s, total: 41min 10s
Wall time: 51min 34s


# Calculate Ignition probability

In [13]:
PGA_SA.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 20336384 entries, 0 to 400841
Data columns (total 6 columns):
 #   Column     Dtype   
---  ------     -----   
 0   event_id   int64   
 1   site_id    int64   
 2   gmv_PGA    float64 
 3   geometry   geometry
 4   SA_IgnPrb  float64 
 5   IgnProb    float64 
dtypes: float64(3), geometry(1), int64(2)
memory usage: 1.1 GB


In [14]:
def SA_Ignition_Probability(merge_data):
    merge_data['SA_IgnPrb'] = 1/(np.exp(-1*(-6.755+8.463+merge_data['gmv_PGA'] + \
                                            0.0000984+merge_data['PopDensity']+ \
                                            0.001523+merge_data['BldgFloo_1'])))
    merge_data['IgnProb'] = 1 - \
        np.power(1-merge_data['SA_IgnPrb'], + 1/merge_data['NumberBldg'])
    return merge_data

In [15]:
%%time
IgnitionData = SA_Ignition_Probability(PGA_SA)

KeyError: 'PopDensity'

# Save to file

In [10]:
%%time
IgnitionData.drop(['lon', 'lat', 'index_right', \
                   'BldgFloo_1', 'PopDensity', 'NumberBldg'], \
                  axis=1, \
                 inplace=True)

KeyError: "['lon' 'lat' 'index_right' 'BldgFloo_1' 'PopDensity' 'NumberBldg'] not found in axis"

In [ ]:
%%time
p_event = Path('/Users/alex/Dropbox/Work/GNS/008_FFE/ProbaFFE/PGA_event')

for event in range(IgnitionData['event_id'].max()):
    try:
        print(f"event {event}")
        i_data = IgnitionData[IgnitionData['event_id']==event]
        i_data.to_file(str(p_event) + f"/event_{event}.shp")
    except:
        print(f"no event {event}")

event 0
event 1


/Users/alex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  import sys


event 2
event 3
event 4
event 5
event 6
event 7
event 8
event 9
event 10
event 11
event 12
event 13
event 14
event 15
event 16
event 17
event 18
event 19
event 20
event 21
event 22
event 23
event 24
event 25
event 26
event 27
event 28
event 29
event 30
event 31
event 32
event 33
event 34
event 35
event 36
event 37
event 38
event 39
event 40
event 41
event 42
event 43
event 44
event 45
event 46
event 47
event 48
event 49
event 50
event 51
event 52
event 53
event 54
event 55
event 56
event 57
event 58
event 59
event 60
event 61
event 62
event 63
event 64
event 65
event 66
event 67
event 68
event 69
event 70
event 71
event 72
event 73
event 74
event 75
event 76
event 77
event 78
event 79
event 80
event 81
event 82
event 83
event 84
event 85
event 86
event 87
event 88
event 89
event 90
event 91
event 92
event 93
event 94
event 95
event 96
event 97
event 98
event 99
event 100
event 101
event 102
event 103
event 104
event 105
event 106
event 107
event 108
event 109
event 110
event 111
event 

event 666
event 667
event 668
event 669
event 670
event 671
event 672
event 673
event 674
event 675
event 676
event 677
event 678
event 679
event 680
event 681
event 682
event 683
event 684
event 685
event 686
event 687
event 688
event 689
event 690
event 691
event 692
event 693
event 694
event 695
event 696
event 697
event 698
event 699
event 700
event 701
event 702
event 703
event 704
event 705
event 706
event 707
event 708
event 709
event 710
event 711
event 712
event 713
event 714
event 715
event 716
event 717
event 718
event 719
event 720
event 721
event 722
event 723
event 724
event 725
event 726
event 727
event 728
event 729
event 730
event 731
event 732
event 733
event 734
event 735
event 736
event 737
event 738
event 739
event 740
event 741
event 742
event 743
event 744
event 745
event 746
event 747
event 748
event 749
event 750
event 751
event 752
event 753
event 754
event 755
event 756
event 757
event 758
event 759
event 760
event 761
event 762
event 763
event 764
event 765


event 1442
event 1443
event 1444
event 1445
event 1446
event 1447
event 1448
event 1449
event 1450
event 1451
event 1452
event 1453
event 1454
event 1455
event 1456
event 1457
event 1458
event 1459
event 1460
event 1461
event 1462
event 1463
event 1464
event 1465
event 1466
event 1467
event 1468
event 1469
event 1470
event 1471
event 1472
event 1473
event 1474
event 1475
event 1476
event 1477
event 1478
event 1479
event 1480
event 1481
event 1482
event 1483
event 1484
event 1485
event 1486
event 1487
event 1488
event 1489
event 1490
event 1491
event 1492
event 1493
event 1494
event 1495
event 1496
event 1497
event 1498
event 1499
event 1500
event 1501
event 1502
event 1503
event 1504
event 1505
event 1506
event 1507
event 1508
event 1509
event 1510
event 1511
event 1512
event 1513
event 1514
event 1515
event 1516
event 1517
event 1518
event 1519
event 1520
event 1521
event 1522
event 1523
event 1524
event 1525
event 1526
event 1527
event 1528
event 1529
event 1530
event 1531
event 1532

event 2188
event 2189
event 2190
event 2191
event 2192
event 2193
event 2194
event 2195
event 2196
event 2197
event 2198
event 2199
event 2200
event 2201
event 2202
event 2203
event 2204
event 2205
event 2206
event 2207
event 2208
event 2209
event 2210
event 2211
event 2212
event 2213
event 2214
event 2215
event 2216
event 2217
event 2218
event 2219
event 2220
event 2221
event 2222
event 2223
event 2224
event 2225
event 2226
event 2227
event 2228
event 2229
event 2230
event 2231
event 2232
event 2233
event 2234
event 2235
event 2236
event 2237
event 2238
event 2239
event 2240
event 2241
event 2242
event 2243
event 2244
event 2245
event 2246
event 2247
event 2248
event 2249
event 2250
event 2251
event 2252
event 2253
event 2254
event 2255
event 2256
event 2257
event 2258
event 2259
event 2260
event 2261
event 2262
event 2263
event 2264
event 2265
event 2266
event 2267
event 2268
event 2269
event 2270
event 2271
event 2272
event 2273
event 2274
event 2275
event 2276
event 2277
event 2278

event 2932
event 2933
event 2934
event 2935
event 2936
event 2937
event 2938
event 2939
event 2940
event 2941
event 2942
event 2943
event 2944
event 2945
event 2946
event 2947
event 2948
event 2949
event 2950
event 2951
event 2952
event 2953
event 2954
event 2955
event 2956
event 2957
event 2958
event 2959
event 2960
event 2961
event 2962
event 2963
event 2964
event 2965
event 2966
event 2967
event 2968
event 2969
event 2970
event 2971
event 2972
event 2973
event 2974
event 2975
event 2976
event 2977
event 2978
event 2979
event 2980
event 2981
event 2982
event 2983
event 2984
event 2985
event 2986
event 2987
event 2988
event 2989
event 2990
event 2991
event 2992
event 2993
event 2994
event 2995
event 2996
event 2997
event 2998
event 2999
event 3000
event 3001
event 3002
event 3003
event 3004
event 3005
event 3006
event 3007
event 3008
event 3009
event 3010
event 3011
event 3012
event 3013
event 3014
event 3015
event 3016
event 3017
event 3018
event 3019
event 3020
event 3021
event 3022

event 3677
event 3678
event 3679
event 3680
event 3681
event 3682
event 3683
event 3684
event 3685
event 3686
event 3687
event 3688
event 3689
event 3690
event 3691
event 3692
event 3693
event 3694
event 3695
event 3696
event 3697
event 3698
event 3699
event 3700
event 3701
event 3702
event 3703
event 3704
event 3705
event 3706
event 3707
event 3708
event 3709
event 3710
event 3711
event 3712
event 3713
event 3714
event 3715
event 3716
event 3717
event 3718
event 3719
event 3720
event 3721
event 3722
event 3723
event 3724
event 3725
event 3726
event 3727
event 3728
event 3729
event 3730
event 3731
event 3732
event 3733
event 3734
event 3735
event 3736
event 3737
event 3738
event 3739
event 3740
event 3741
event 3742
event 3743
event 3744
event 3745
event 3746
event 3747
event 3748
event 3749
event 3750
event 3751
event 3752
event 3753
event 3754
event 3755
event 3756
event 3757
event 3758
event 3759
event 3760
event 3761
event 3762
event 3763
event 3764
event 3765
event 3766
event 3767